In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Cargando Datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
df_2015 = pd.read_csv("/kaggle/input/world-happiness/2015.csv")
df_2015

df_2016 = pd.read_csv("/kaggle/input/world-happiness/2016.csv")
df_2016

df_2017 = pd.read_csv("/kaggle/input/world-happiness/2017.csv")
df_2017

df_2018 = pd.read_csv("/kaggle/input/world-happiness/2018.csv")
df_2018

df_2019 = pd.read_csv("/kaggle/input/world-happiness/2019.csv")
df_2019

In [ ]:
print(df_2015.shape)
print(df_2016.shape)
print(df_2017.shape)
print(df_2018.shape)
print(df_2019.shape)

In [ ]:
df_2015.columns.to_list()

In [ ]:
df_2016.columns.to_list()

In [ ]:
df_2017.columns.to_list()

In [ ]:
df_2018.columns.to_list()

In [ ]:
df_2019.columns.to_list()

Los datos del 2018 y 2019 tienen las mismas variables, además de que son las más actuales, debido a eso trataremos con estas 2 bases de datos para el análisis.

In [ ]:
df = pd.DataFrame()
df = df_2018.copy()
df = df.append(df_2019,ignore_index=True)
df

In [ ]:
df.columns.to_list()

In [ ]:
df.columns = ['Ranking','País o Región','Puntaje','PIB per cápita','Apoyo Social','Expectativa de vida saludable','Libertad para tomar decisiones en la vida','Generosidad','Percepción de Corrupción']
df.columns.to_list()

In [ ]:
df.shape

# 1. Introducción

 En el dataset a estudiar contamos con las siguientes variables:
 
* Ranking
* País o Región
* Puntaje
* PIB per cápita
* Apoyo Social
* Expectativa de vida saludable
* Libertad para tomar decisiones en la vida
* Generosidad
* Percepción de Corrupción

Ahora para analizar sobre la felicidad de la población, muchas veces asumimos variables que podrían influir en la felicidad de la población como la generosidad de las personas, la corrupción percibida o la libertad para tomar desiciones en la vida, pero en este análisis demostraremos que tal vez la intuición puede fallarnos y demostraremos con números las variables más influyentes sobre la felicidad de las personas y además encontraremos variables que intuitivamente pensaríamos que influye en la felicidad, pero en la realidad no lo hace.

<img src="https://imagenes.iberoeconomia.es/wp-content/uploads/2018/03/20125134/Paises-m%C3%A1s-felices-de-Am%C3%A9rica-Latina--737x413.jpg">

# 2. Análisis sobre el Dataset

### Tipos de datos de todas las características

En este dataset contamos con 8 variables numéricas y 1 categórica.

In [ ]:
df.dtypes

### Veremos los valores únicos de cada variables

Analizamos los valores únicos

In [ ]:
for i in df.columns:
    print("*"*80)
    print("*"*80)
    print(i)
    print(df[i].sort_values().unique())

### Distribución de datos del Puntaje

In [ ]:
datos_x = df.Puntaje.value_counts().index.to_list()
datos_y = df.Puntaje.value_counts().to_list()

plt.title('Distribución de datos de Puntaje')
sns.barplot(x=datos_x, y=datos_y)
plt.show()

# 3. Limpieza de Datos

In [ ]:
df.isnull().sum()

In [ ]:
df[df.isnull().any(1)]

In [ ]:
df[df['País o Región']=='United Arab Emirates']

Imputaremos la percepción de corrupción faltante que pertenece al país Emiratos Árabes Unidos, con un valor perteneciente a otro ranking, ya que estimamos que dicho valor no tendría un cambio muy grande respecto al otro.

In [ ]:
df_null = pd.DataFrame()
df_null = df.copy()

df_null.loc[df_null['País o Región']=='United Arab Emirates','Percepción de Corrupción'] = 0.182
df_null[df_null['País o Región']=='United Arab Emirates']

# 4. Análisis Exploratorio de Datos (AED)

## Matriz de gráfico multivariado

Generamos un gráfico multivariado, en el que no contaremos con las variables 'Overall rank' y 'Country or region', debido a que la primera se derivaría de la variable 'Score' y la segunda haría ininteligible el gráfico debido a que es una variable categórica.

In [ ]:
import plotly.express as px
fig = px.scatter_matrix(df_null.drop(['País o Región','Ranking'],axis=1),
                        color="Puntaje", height=1300)
fig.show()

## Matriz de Correlación

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df_null.corr(), annot = True, cmap="YlGnBu")
plt.show()

### Datos Interesantes:
* **Puntaje vs PIB per cápita**: La relación más significativa es esta, la felicidad percibida por una población se encuentra muy relacionada por el PIB per cápita de un país, esto puede ser debido a que una mejor economía indicaría mejores condiciones de vida para la población de un país.
* **Puntaje vs Apoyo Social**: Otra de las variables que pesa mucho en la felicidad de las personas, es el apoyo social que reciben de parte de su entorno, es decir, que se encuentran muy estimulados por el apoyo de sus amigos, familiares, entorno laboral, etc.
* **Puntaje vs Expectativa de vida saludable**: Esta variable se encuentra igual de relacionada que el "Apoyo Social", la felicidad está fuertemente relacionada a la percepción que tiene una población, acerca de su expectativa de vida saludable.
* **Puntaje vs Libertad para tomar desiciones en la vida**: La influencia entre ambas variables es moderada, es decir, aún no se puede dar una afirmación del por qué de la relación entre ambas variables, ya que paises con más libertad de elección sobre sus vidas, tienen puntajes bajos como Bolivia.

### Datos a agregar:
* **PIB per cápita vs Expectativa de vida saludable**: El PIB per cápita esta fuertemente relacionado por la expectativa de vida saludable de las personas. La expectativa de vida saludable puede ser una consecuencia del PIB per cápita, ya que una mejor producción podría deberse a buenos trabajos en mejores condiciones y mejor pagados, lo que conllevaría a que las personas pudieran optar por vidas más saludables.
* **PIB per cápita vs Apoyo Social**: Mejores oportunidades laborales, la tranquilidad de tener una economía más estable, influirían en el estado emocional de las personas, lo que conllevaría en brindar un mejor apoyo social. 
* **Apoyo Social vs Expectativa de vida saludable**: La relación entre ambas variables pudiera deberse a que un ambiente donde se sienta el apoyo social, propiciaría una buena salud mental, con una mejor expectativa sobre la vida, aumentando la expectativa de vida saludable.

### Nota:
* La felicidad no está influenciada (0.11) por la generosidad de la gente del país.
* La felicidad está muy poco influenciada (0.4) por la percepción de corrupción que tiene la gente sobre el país que reside.

## Gráfico en 3D de las variables más importantes

In [ ]:
import plotly.express as px

fig = px.scatter_3d(df_null, x='Apoyo Social', y='Expectativa de vida saludable', z='PIB per cápita',
                    color='Puntaje', opacity=0.7)
fig.show()

# 5. Gráfico de Mapas

## Preparando los datos

In [ ]:
df_map = pd.DataFrame()
df_map = df_2019.copy()

In [ ]:
import pycountry

def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
    except Exception:
        return np.nan
    else:
        return result[0].alpha_3
    
df_map.loc[:,'country_code'] = df_map["Country or region"].apply(lambda country: do_fuzzy_search(country))

In [ ]:
df_map.isnull().sum()

In [ ]:
df_map[df_map.isnull().any(1)].sort_values('Country or region')

In [ ]:
df_map.loc[df_map['Country or region']=='South Korea','country_code'] = 'KR'
df_map.loc[df_map['Country or region']=='Laos','country_code'] = 'LA'
df_map.loc[df_map['Country or region']=='Trinidad & Tobago','country_code'] = 'TT'
df_map.loc[df_map['Country or region']=='Ivory Coast','country_code'] = 'CI'
df_map.loc[df_map['Country or region']=='Palestinian Territories','country_code'] = 'PS'
df_map.loc[df_map['Country or region']=='Northern Cyprus','country_code'] = 'CY'
df_map.loc[df_map['Country or region']=='Swaziland','country_code'] = 'SZ'
df_map.loc[df_map['Country or region']=='Congo (Brazzaville)','country_code'] = 'CG-BZV'
df_map.loc[df_map['Country or region']=='Congo (Kinshasa)','country_code'] = 'CD-KN'

In [ ]:
df_map.columns = ['Ranking','País o Región','Puntaje','PIB per cápita','Apoyo Social','Expectativa de vida saludable','Libertad para tomar decisiones en la vida','Generosidad','Percepción de Corrupción','country_code']
df_map.columns.to_list()

## Graficando mapa

In [ ]:
import plotly.graph_objects as go

def mapa(country_code, score, country_name):
    fig = go.Figure(data=go.Choropleth(
        locations = country_code,
        z = score,
        text = country_name,
        colorscale = 'dense',
        autocolorscale=False,
        reversescale=False,
        marker_line_color='black',
        marker_line_width=0.5,
        colorbar_tickprefix = '',
        colorbar_title = score.name,
    ))
    
    fig.update_layout(
        title_text='Ranking de Felicidad 2019 ('+str(score.name)+')',
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='equirectangular'
        ),
        annotations = [dict(
            x=0.55,
            y=0.1,
            xref='paper',
            yref='paper',
            text=' ',
            showarrow = False
        )]
    )
    fig.show()
    
    return 

In [ ]:
mapa(df_map['country_code'],df_map['Puntaje'],df_map['País o Región'])

### El PIB per cápita se encuentra muy relacionado a la felicidad de un país, entonces si lo comparamos con el mapa de puntaje, veremos que ambos tienen mucho parecido.

In [ ]:
mapa(df_map['country_code'],df_map['PIB per cápita'],df_map['País o Región'])

### La felicidad no está influenciada por la generosidad!

In [ ]:
mapa(df_map['country_code'],df_map['Generosidad'],df_map['País o Región'])

### La felicidad no está influenciada por la percepción de corrupción del país! De hecho, países con alto puntaje en felicidad perciben alta corrupción en sus países como Finlandia, Suecia, Noruega, etc.

In [ ]:
mapa(df_map['country_code'],df_map['Percepción de Corrupción'],df_map['País o Región'])

In [ ]:
mapa(df_map['country_code'],df_map['Libertad para tomar decisiones en la vida'],df_map['País o Región'])

# 6. Conclusiones

No hay duda de que el **PIB per cápita**, **Apoyo Social** y **Expectativa de vida saludable**, son factores clave para la felicidad de un país, luego le sigue la libertad para tomar desiciones en la vida que tiene influencia moderada sobre la felicidad de un país.

Por otro lado, poniendo en contra nuestra intuición la **generosidad** y **la percepción de corrupción** tienen muy poca influencia sobre la felicidad de las personas, de hecho países con altos índices de felicidad como Finlandia, Noruega, etc. Han tenido pocos índices en **generosidad** y **alta percepción de corrupción** sobre su país.

<img src="https://diariouno.pe/wp-content/uploads/2016/03/1412.jpg">